<a href="https://colab.research.google.com/github/Markohf/Titanic/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pre-trabajo

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [28]:
train0 = '/content/drive/MyDrive/Colab Notebooks/Data Science - Coding Dojo/Proyecto 2/Datos/titanic_train.csv'
train0 = pd.read_csv(train0)
test0 = '/content/drive/MyDrive/Colab Notebooks/Data Science - Coding Dojo/Proyecto 2/Datos/titanic_test.csv'
test0 = pd.read_csv(test0)
result0 = '/content/drive/MyDrive/Colab Notebooks/Data Science - Coding Dojo/Proyecto 2/Datos/gender_submission.csv'
result0 = pd.read_csv(result0)

# Limpizar y Analizar Data

In [29]:
train0.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [30]:
train0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [31]:
train0.duplicated().sum()

0

In [32]:
train1=train0

In [33]:
sex_ = train1['Sex']
sex_ = pd.get_dummies(sex_,prefix='sex')
embarked_ = train1['Embarked']
embarked_ = pd.get_dummies(embarked_,prefix='embarked')
train1 = train1.drop(columns=['PassengerId','Name','Sex','Age','Ticket','Cabin','Embarked'])
train1 = pd.concat([sex_,embarked_,train1],axis = 1)

In [34]:
train1.head()

,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,Survived,Pclass,SibSp,Parch,Fare
0,0,1,0,0,1,0,3,1,0,7.2500
1,1,0,1,0,0,1,1,1,0,71.2833
2,1,0,0,0,1,1,3,0,0,7.9250
3,1,0,0,0,1,1,1,1,0,53.1000
4,0,1,0,0,1,0,3,0,0,8.0500


#Modelo

In [35]:
train2 = train1

In [36]:
X = train2.drop(columns=['Survived']).values
y = train2['Survived'].values

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

In [46]:
forest_clf = RandomForestClassifier()
param_grid = [{'max_depth':[4,5,6],'n_estimators':[40,50,60]}]
grid_search = GridSearchCV(forest_clf, param_grid, cv=5,
                          return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid=[{'max_depth': [4, 5, 6],
                          'n_estimators': [40, 50, 60]}],
             return_train_score=True)

In [47]:
cvres = grid_search.cv_results_
# zip junta parámetros en dos listas como uno a uno
for values in zip(cvres['mean_test_score'], cvres['params']):
    print(values) # Mostramos el accuracy y el n_estimator en cada prueba de validación

(0.8033881611346401, {'max_depth': 4, 'n_estimators': 40})
(0.7991825076332119, {'max_depth': 4, 'n_estimators': 50})
(0.8089727174234216, {'max_depth': 4, 'n_estimators': 60})
(0.8005614104205654, {'max_depth': 5, 'n_estimators': 40})
(0.8117896188318724, {'max_depth': 5, 'n_estimators': 50})
(0.814606520240323, {'max_depth': 5, 'n_estimators': 60})
(0.8005811090318133, {'max_depth': 6, 'n_estimators': 40})
(0.8132079188417217, {'max_depth': 6, 'n_estimators': 50})
(0.8103910174332709, {'max_depth': 6, 'n_estimators': 60})


In [48]:
forest_clf = RandomForestClassifier(n_estimators = 50, max_depth = 5)
forest_clf.fit(X_train, y_train)
forest_clf.score(X_test, y_test)

0.7821229050279329